### 1. 변수 생성

In [ ]:
import time
import re

import bareunpy
from bareunpy import Tagger
from jamo import h2j, j2hcj
import nltk
from nltk.tokenize import sent_tokenize 
from konlpy.tag import Komoran
komoran = Komoran()

import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
### 필요한 데이터 불러오기

# 전처리된 신문 기사 텍스트
data = pd.read_csv('변수포함 데이터셋_6차_9520.csv', encoding = 'utf-8-sig')

# 모두의 말뭉치 일상 빈도
corp_freq = pd.read_csv('corpus_frequency.csv', encoding = 'utf-8-sig')
corp_freq.drop(['Unnamed: 0'], inplace = True, axis = 1)

# 물결 21 빈도
wave_corp = pd.read_csv('물결21_총빈도수_최종.csv', encoding='utf-8-sig')
wave_corp.drop(['Unnamed: 0.1', 'Unnamed: 0'], inplace = True, axis = 1)

In [ ]:
# 일상에서 자주 쓰이는지를 확인하기 위한 변수이기 때문에 quantile 확인 후, 빈도수로 잘라야함
q1 = np.percentile(corp_freq['frequency'], 25)
median = np.median(corp_freq['frequency'])
q3 = np.percentile(corp_freq['frequency'], 75)
mean = np.mean(corp_freq['frequency'])
variance = np.var(corp_freq['frequency'])
std_dev = np.std(corp_freq['frequency'])
print('1사분위 수 : {}, 중위값 : {}, 3사분위 수 : {}, 평균 {}, 분산 {}, 표준편차 : {}'.format(q1, median, q3, mean, variance, std_dev))

# 평균 이상으로 일상 빈도 업데이트
daily_usage = corp_freq[corp_freq['frequency'] >= mean]

#### (1) 음운론적 8단계

In [ ]:
class ComplexityAnalyzer:
    
    def __init__(self, already_know=[]):
        self.already_know = already_know # 학습단어 연동
        self.CHO_basic = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅎ'] 
        self.CHO_advanced = ['ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ'] 
        self.JOONG_single = ['ㅏ', 'ㅓ', 'ㅗ', 'ㅜ', 'ㅐ', 'ㅔ', 'ㅡ', 'ㅣ'] 
        self.JOONG_double = ['ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅒ', 'ㅖ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅞ', 'ㅝ', 'ㅟ', 'ㅢ'] 
        self.JONG = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']

        self.weight_list = [] # 가중치 담을 리스트 준비
        self.level_weight = None # 가중치 초기화

        self.already_words = [] # 문장 내 학습 단어 담을 리스트 준비
        
        
    def syllable_tokenizer(self, s):
        temp = s.replace(' ', '').replace('\n', '').replace('"', '')
        result = []
        for c in temp:
            result.append(c)
        return result
    
    def normalizer(self, value): # 가중치 때문에 최저 0.5까지 나올 수 있음!
        return (value - 0.5) / (8 - 1)
    
    def text_level(self, text):
        # 음절단위 분리
        text_syl = self.syllable_tokenizer(text)
        
        # 음소단위 분리
        pho_list = []
        for syl in text_syl:
            phoneme = j2hcj(h2j(syl))
            pho_list.append(phoneme)
        
        # complexity 판별
        level_list = []
        
        def jong_no(word):
            if word[0] in self.CHO_basic and word[1] in self.JOONG_single:
                level = 1
                level_list.append(level)
            elif word[0] in self.CHO_advanced and word[1] in self.JOONG_single:
                level = 2
                level_list.append(level)
            elif word[0] in self.CHO_basic and word[1] in self.JOONG_double:
                level = 5
                level_list.append(level)
            elif word[0] in self.CHO_advanced and word[1] in self.JOONG_double:
                level = 7
                level_list.append(level)
            else :
                level = np.NAN
                level_list.append(level)
        
        def jong_yes(word):
            if word[0] in self.CHO_basic and word[1] in self.JOONG_single and word[2] in self.JONG:
                level = 3
                level_list.append(level)
            elif word[0] in self.CHO_advanced and word[1] in self.JOONG_single and word[2] in self.JONG:
                level = 4
                level_list.append(level)
            elif word[0] in self.CHO_basic and word[1] in self.JOONG_double and word[2] in self.JONG:
                level = 6
                level_list.append(level)
            elif word[0] in self.CHO_advanced and word[1] in self.JOONG_double and word[2] in self.JONG:
                level = 8
                level_list.append(level)
            else :
                level = np.NAN
                level_list.append(level)
        
        for pho in pho_list:
            if len(pho) == 2:
                jong_no(pho)
            elif len(pho) == 3:
                jong_yes(pho)
                
        ## [NEW] 가중치 반영
        if self.level_weight is not None : # 가중치 리스트 있는 경우에만!
            #print('level weight', self.level_weight) # 가중치 확인하고 싶으면 '#'을 해제하세요
            level_list = [a * b for a, b in zip(level_list, self.level_weight)] # 레벨리스트에 가중치리스트 곱해서 최종 레벨리스트 생성!
        else :
            #print("level list:", level_list) # 난이도 확인하고 싶으면 '#'을 해제하세요
            pass
        
        
        if len(level_list) == 0:
            #print(text, "---> 난이도 불필요 문장") # 난이도 불필요 문장을 확인하고 싶으면 '#'을 해제하세요
            return 0, level_list # 잘못된 경우 0(zero)을 반환 (NaN 값 처리)
        
        syl_len = len(level_list)  # 음절 수
        level_sum = sum(level_list)  # complexity 총합
        score = level_sum / syl_len
        
        score_scaled = self.normalizer(score)  # 스케일링
        score_per = round(score_scaled * 100, 1)  # 점수화
        
        # 어려운 단어 추출에 활용하기 위해 level_list도 함께 반환!
        return score_per, level_list
    
    def analyze_syl(self, text): ##음절단위 분석기##
        sent_2 = text
        score_list1 = []
        weight_list = None # 가중치리스트 초기화
        
        score_t = self.text_level(sent_2)

        if score_t == 0 :
            score_t = 0
            score_list1.append(score_t)
        else:
            score_t = score_t[0] # score 값만 사용할 거라서 인덱싱
            score_list1.append(score_t)
                
        
        df1 = pd.DataFrame({'문장 내용': sent_2, '음절단위 점수': score_list1})
        return df1
    
    def analyze_noun(self, text): ##명사단위 분석기##
        sent_2 = text
        score_list2 = []
        difficult_list = []  # 어려운 음절 저장할 리스트
        
        # 명사만 뽑아낸 뒤에,,,
        sent_2_noun = []
        for t1 in sent_2 :
            nouns_2 = komoran.nouns(t1)
            sent_2_noun.append(nouns_2)
            
        # ,,, 다시 입력 형태에 맞게 문자열로 이어붙이기!  
        noun_2_str = []
        for t2 in sent_2_noun : 
            
            weights, words = self.weight_maker(t2) # 가중치 만드는 함수를 거치는 구간 
            self.weight_list.append(weights)
            self.already_words.append(words)
            
            comb_nouns = ' '.join(t2) # 문자열 이어붙이는 구간
            noun_2_str.append(comb_nouns)
        
        # 그리고 그걸 다시 text_level에 입력!
        score_list2 = []

        for idx, t3 in enumerate(noun_2_str) : 
            self.level_weight = self.weight_list[idx] # 해당 문장에 맞는 가중치 리스트 꺼내기
            score_t, levels = self.text_level(t3)
            if score_t == 0 :
                score_t = 0
                score_list2.append(score_t)
            else:
                
                score_list2.append(score_t)
                
                 # [NEW] 어려운 단어 추출
                
            diff_syls = []            
            for ldx, level in enumerate(levels):
                t3 = t3.replace(' ', '').replace('\n', '').replace('"', '') # 길이 맞추기 위해 동일하게 전처리해주고
                if level >= 5:
                    diff_t3 = "[{}]{}".format(ldx, t3[ldx])
                    diff_syls.append(diff_t3)
                else:
                    diff_syls.append(' ')
            
            diff_syls = list(dict.fromkeys(diff_syls)) # 중복 제거 (set은 순서가 뒤섞여서 사용 취소)
            diff_syls = '  '.join(diff_syls) # 보기 좋게 대괄호 제거
                    
            difficult_list.append(diff_syls) # 문장 돌면서 '문장별 어려운 음절' 추가
            
        already_words_set = [', '.join(lst) for lst in self.already_words] # 학습한 단어들 출력 (깔끔하게 대괄호 제거)
        
        
        df2 = pd.DataFrame({'문장 내용': sent_2, '명사단위 점수': score_list2, '난이도 5 이상 음절': difficult_list, '학습한 단어': already_words_set})
        return df2

    def weight_maker(self, nounset) : 
        weight_already = []
        word_already = []
        
        for noun in nounset : 

            if noun in self.already_know : # 만약 이미 학습한 단어라면,
                word_already.append(noun) # (ㄱ) 학습한 단어 목록에 추가
                for i in range(len(noun)) : # (ㄴ) 가중치 목록에 음절 수만큼 가중치 추가
                    weight_already.append(0.5)
            else :
                for i in range(len(noun)) :
                    weight_already.append(1)
        
        return weight_already, word_already # 생성된 가중치리스트를 반환해주자!

In [ ]:
CHO_basic = ['ㄱ', 'ㄴ', 'ㄷ', 'ㄹ', 'ㅁ', 'ㅂ', 'ㅅ', 'ㅇ', 'ㅈ', 'ㅎ'] # 기본자음 10개
CHO_advanced = ['ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㄲ', 'ㄸ', 'ㅃ', 'ㅆ', 'ㅉ'] # 격음 또는 경음 9개

JOONG_single = ['ㅏ', 'ㅓ', 'ㅗ', 'ㅜ', 'ㅐ', 'ㅔ', 'ㅡ', 'ㅣ'] # 단모음 8개
JOONG_double = ['ㅑ', 'ㅕ', 'ㅛ', 'ㅠ', 'ㅒ', 'ㅖ', 'ㅘ', 'ㅙ', 'ㅚ', 'ㅞ', 'ㅝ', 'ㅟ', 'ㅢ'] # 이중모음 13개

JONG = ['ㄱ', 'ㄲ', 'ㄳ', 'ㄴ', 'ㄵ', 'ㄶ', 'ㄷ', 'ㄹ', 'ㄺ', 'ㄻ', 'ㄼ', 'ㄽ', 'ㄾ', 'ㄿ', 'ㅀ', 'ㅁ', 'ㅂ', 'ㅄ', 'ㅅ', 'ㅆ', 'ㅇ', 'ㅈ', 'ㅊ', 'ㅋ', 'ㅌ', 'ㅍ', 'ㅎ']


def syllable_tokenizer(s):
    temp = s.replace(' ', '').replace('\n', '').replace('"', '') # 불필요한 기호 및 줄바꿈 삭제
    result = []
    for c in temp: # 입력받은 temp를 인덱싱으로 돌면서
        result.append(c) # 한 음절씩 결과 리스트에 추가
    return result

def normalizer(value) : # score값들을 정규화해주는 함수를 정의
    return (value - 1) / (8 - 1) # 최소1 최대8의 값을 0~1 사이로 정규화

In [ ]:
def text_level(text) :
    
     # (1) 음절단위 분리
    text_syl = syllable_tokenizer(text)
    
    # (2) 음소단위 분리
    pho_list=[]
    for syl in text_syl :
        phoneme = j2hcj(h2j(syl))
        pho_list.append(phoneme)
    
    # (3) 복잡도 판별
    level_list = []
    # 함수 설정 (깔끔한 출력을 위해 불필요한 print문 전부 제거함)
    def jong_no(word):  
        if word[0] in CHO_basic and word[1] in JOONG_single :
            level = 1
            level_list.append(level)
        elif word[0] in CHO_advanced and word[1] in JOONG_single :
            level = 4
            level_list.append(level)
        elif word[0] in CHO_basic and word[1] in JOONG_double :
            level = 25
            level_list.append(level)
        elif word[0] in CHO_advanced and word[1] in JOONG_double :
            level = 49
            level_list.append(level)
        else :
            pass
        
    def jong_yes(word):
        if word[0] in CHO_basic and word[1] in JOONG_single and word[2] in JONG :
            level = 9
            level_list.append(level)
        elif word[0] in CHO_advanced and word[1] in JOONG_single and word[2] in JONG :
            level = 16
            level_list.append(level)
        elif word[0] in CHO_basic and word[1] in JOONG_double and word[2] in JONG :
            level = 36
            level_list.append(level)
        elif word[0] in CHO_advanced and word[1] in JOONG_double and word[2] in JONG :
            level = 64
            level_list.append(level)
        else :
            pass
    
    for pho in pho_list : # (2)의 결과 pho_list에 대해서
        if len(pho) == 2 : # 받침이 없는 경우는 jong_no 함수로
            jong_no(pho)
        elif len(pho) == 3 : # 받침이 있는 경우는 jong_yes 함수로
            jong_yes(pho)
            
    #print(level_list) 레벨 리스트 완성
    
    # 문장이 아예 문장부호로만 이루어진 경우 등 level_list에 아무것도 없는 경우는 제외가 필요함
    if len(level_list)==0 :
        print(text, "---> 난이도 불필요 문장")
        pass
    
    else :      
        # (4) 평균복잡도 산출
        syl_len = len(level_list) # 음절 수 
        #print(syl_len)
        level_sum = sum(level_list) # 복잡도 총합
        #print(level_sum)
        score = level_sum / syl_len

        # (5) 스케일링 및 점수화
        score_scaled = normalizer(score) # normalizer 함수에 score 입력
        score_per = round(score_scaled*100, 1) # 100을 곱해서 점수화

        # (6) 정규화 안 하고 그냥 출력!
        return score 

In [ ]:
# 데이터셋에 적용
data['음운론적 복잡도'] = data['본문'].apply(text_level)

#### (2) 바른 형태소 분석기를 사용한 변수

In [ ]:
# Tagger 정의
API_KEY="[바른 형태소 분석기 api key]" 
my_tagger = Tagger(API_KEY, 'localhost')

In [ ]:
# 필요한 데이터 불러오기

# 취합본 데이터
data = pd.read_csv('~~취합본 데이터~~', encoding='utf-8-sig')

# 일상빈도 데이터
daily_corp = pd.read_csv('~~일상빈도 데이터~~', encoding='utf-8-sig')

# 물결21 데이터
wave_corp = pd.read_csv('~~물결21 데이터~~', encoding='utf-8-sig')

# 기초어휘 데이터
basic_corp = pd.read_csv('~~기초어휘 데이터~~', encoding='utf-8-sig')

In [ ]:
# 취합본 데이터에서 필요한 부분만 추출
data = data[['언론사', '본문']]

# 일상빈도 데이터에서 필요한 부분만 추출
daily_corp = daily_corp[daily_corp['frequency'] >= np.mean(daily_corp['frequency'])]

# 물결21 데이터에서 필요한 부분만 추출
wave_corp = wave_corp[['태그', '단어', '총 빈도수']]

In [ ]:
# 필요한 품사 태그 지정 -> 수식언, 연결어미, 전성어미
susik_set = ['MMA', 'MAG', 'MAJ']
junsung_set = ['ETN', 'ETM']
yeongyeol_set = 'EC'
important_pos_list = ["NNG", 'NNP', "NNB", "NP", "NR", "NF", "NA", "NV", "VV", "VA", "VX", "VCP", "VCN", "MMA", "MMD", "MMN", "MAG", "MAJ"]

In [ ]:
# 시작 시간 기록
start_time = time.time()

# 값을 저장할 리스트
average_usage_list = []
count_list = []
susik_list = []
junsung_list = []
yeongyeol_list = []
s_length_list = []
wave_list = []
basic_list = []

# '본문'에 대한 처리 시간과 진행 상황을 모니터링
for i, line in enumerate(data['본문'], start=1):
    res = my_tagger.tags([line])
    pa = res.pos() # 일상 빈도
    ma = res.morphs() # 물결 21
    
    # 문장 개수 확인을 위한 tagger(auto_split의 유무가 영향을 주는가?)
    res = my_tagger.tags([line], auto_split=True) # 문장 개수
    m = res.msg()

    # 'pa' 리스트에 있는 형태소의 총 빈도수 합산
    total_usage_pa = 0

    for word, _ in pa:
        if word in daily_corp['corpus'].values:
            frequency = daily_corp.loc[daily_corp['corpus'] == word, 'frequency'].sum()
            total_usage_pa += frequency
            
    # 기초 어휘 count를 위한 'pa' 리스트에 있는 형태소의 총 빈도수 합산
    total_usage_basic = 0

    for word, pos in pa:
        if pos in important_pos_list:
            if word in basic_corp['어휘'].values:
                match_vocab = basic_corp[basic_corp['어휘'] == word]
                if len(match_vocab) == 1:
                    rank = sum(match_vocab["등급"].values)
                else:
                    rank = np.mean(match_vocab["등급"].tolist())
            else:
                rank = 4
        else:
            rank = 0
        total_usage_basic += rank
    
    # 기초어휘점수를 리스트에 추가
    basic_list.append(total_usage_basic)
            
    # 'ma' 리스트에 있는 형태소의 총 빈도수 합산
    total_usage_ma = 0

    for morpheme in ma:
        if morpheme in wave_corp['단어'].values:
            frequency = wave_corp.loc[wave_corp['단어'] == morpheme, '총 빈도수'].sum()
            total_usage_ma += frequency

    # 중복을 포함한 형태소의 개수로 나누어 평균 일상 빈도수 계산
    word_count = len(pa)
    daily_average_usage = total_usage_pa / word_count

    # 계산된 일상 빈도를 리스트에 추가
    average_usage_list.append(daily_average_usage)
    
    # 중복을 포함한 형태소의 개수로 나누어 평균 물결21 빈도수 계산
    morpheme_count = len(ma)
    wave_average_usage = total_usage_ma / morpheme_count

    # 계산된 물결21 빈도수를 리스트에 추가
    wave_list.append(wave_average_usage)

    # 형태소 개수를 리스트에 추가
    count_list.append(word_count)
    
    # 수식언 개수
    filtered_susik = [word for word, pos in pa if pos in susik_set]
    count_susik = len(filtered_susik)
    # 수식언 개수를 리스트에 추가
    susik_list.append(count_susik)
    
    # 전성어미 개수
    filtered_junsung = [word for word, pos in pa if pos in junsung_set]
    count_junsung = len(filtered_junsung)
    # 전성어미 개수를 리스트에 추가
    junsung_list.append(count_junsung)
    
    # 연결어미 개수
    filtered_yeongyeol = [word for word, pos in pa if pos == yeongyeol_set]
    count_yeongyeol = len(filtered_yeongyeol)
    # 연결어미 개수를 리스트에 추가
    yeongyeol_list.append(count_yeongyeol)
    
    # 문장 개수를 count 후 리스트에 추가
    length = len(m.sentences)
    s_length_list.append(length)
    
    
    # i가 10에 도달할 때마다 완료 메시지와 경과 시간 출력
    if i % 10 == 0:
        elapsed_time = time.time() - start_time
        print(f"Processed {i} samples. Elapsed time: {elapsed_time:.2f} seconds")

# 전체 실행 시간 출력
total_time = time.time() - start_time
print(f"Total elapsed time: {total_time:.2f} seconds")

In [ ]:
# 데이터에 추가
data['바른 문장 개수'] = s_length_list
data['바른 형태소 개수'] = count_list

data['일상사용빈도'] = average_usage_list
data['물결21'] = wave_list
data['바른 수식언 개수'] = susik_list
data['바른 전성어미 개수'] = junsung_list
data['바른 연결어미 개수'] = yeongyeol_list
data['기초어휘난이도'] = basic_list

#### (3) 음절 개수

In [ ]:
# 음절 개수 count하는 함수
def count_characters(text):
    cleaned_text = re.sub(r'[^\w\s]', '', text)
    words = cleaned_text.split()
    total_character_count = sum(len(word) for word in words)
    return total_character_count

In [ ]:
# 데이터에 적용
data['음절 개수'] = data['본문'].apply(count_characters)

In [ ]:
# 단어 개수 세기
def count_words(text):
    words = text.split()
    num_words = len(words)
    return num_words

In [ ]:
data['음절 개수'] = data['본문'].apply(count_words)

#### (4) 파생 변수 생성

In [ ]:
# 평균 문장 길이
data['평균 문장 길이'] = data['단어 개수'] / data['바른 문장 개수']

# 평균 단어 길이
data['평균 단어 길이'] = data['음절 개수'] / data['음절 개수']

#### (5) 명사 난이도 파악 후 어려운 어휘 추출

In [ ]:
#------------------------------# 바른으로 품사 태깅
desired_pos_tags = ['NNG']

# 시작 시간 기록
start_time = time.time()

# 값을 저장할 리스트
score_column = []

# '본문'에 대한 처리 시간과 진행 상황을 모니터링
for i, line in enumerate(data['본문'], start=1):
    
    score_list = []
    
    res = my_tagger.tags([line])

    # 'ma' 리스트에 있는 형태소의 총 빈도수 합산
    total_usage = 0
    
    filtered_result = [(word, pos) for word, pos in res.pos() if pos in desired_pos_tags]
    tokens = [word for word, _ in filtered_result]
    
    for token in tokens:
        daily_freq = daily_usage.loc[daily_usage['corpus'] == token, 'frequency'].mean()
        wave_freq = wave_corp.loc[wave_corp['단어'] == token, '총 빈도수'].mean()
        if np.isnan(daily_freq):
            score = wave_freq / 1
        else:
            score = wave_freq / daily_freq
        
        score_tuple = (token, score) # 토큰과 score를 tuple 형태로 저장하고
        score_list.append(score_tuple) # 리스트에 하나씩 저장
    
    score_column.append(score_list) # for문 다 돌아간 리스트들을 다시 최종 리스트 내에 저장
    
    if i % 10 == 0:
        elapsed_time = time.time() - start_time
        print(f"Score Calculating Processed {i} samples. Elapsed time: {elapsed_time:.2f} seconds")

# 데이터에 추가
data['점수_tuple'] = score_column

#------------------------------# 단어만 뽑아내기
total_word_list = []

for i in range(len(data)):
    line = data.iloc[i, 2]
    word_list = []
    for (word, num) in line:
        word_list.append(word)
    total_word_list.append(word_list)
    
#------------------------------# Counter 함수로 빈도수(count)를 반영해보자!
total_result_list = []

for l in total_word_list:
    asd = Counter(l)
    word_frequency_pairs = list(asd.items())
    result_list = []
    result_list.extend(word_frequency_pairs)
    total_result_list.append(result_list)
    
# 데이터에 추가
data['counter'] = total_result_list

#------------------------------# 점수와 빈도수를 곱하기
result_list = []

for i in range(len(data)): # 데이터셋의 모든 행에 대해
    list1 = data.iloc[i, 2] # '점수_tuple' 열에 접근
    list2 = data.iloc[i, 3] # 'counter' 열에 접근
    
    new_tuple_list = []

    # 첫번째 리스트를 기준으로 루프
    for tup1 in list1:        
        # 튜플에서 단어 추출
        word = tup1[0]

        # 두번째 리스트에서 해당 단어에 대한 튜플 찾기
        matching_tup2 = next((tup for tup in list2 if tup[0] == word), None)

        # 만약에 찾은 경우
        if matching_tup2 is not None:
            # nan과 숫자를 곱해 새로운 튜플 생성
            new_tuple = (word, matching_tup2[1] * tup1[1])
            new_tuple_list.append(new_tuple)
            
    # 결과 리스트에 새 튜플리스트들을 축적!
    result_list.append(new_tuple_list)
    
# 데이터에 추가
data['단어난이도'] = result_list

#------------------------------# (단어, 빈도수) 쌍에 대해서 nan 제거, 중복 제거, 빈도수 내림차순 정렬 그리고 불용어 제거
stopwords = ['요구', '특파원', '참석자', '희생자', '기자', '지난해', '양국', '갑', '을', '중대형', '승용차', '이듬해', '핸드볼',
             '국가', '연합뉴스', '당국', '지난해', '기업', '상승세', '닷새', '누리집', '꼴찌', '사망자', '이날', '대통령', '지역',
             '시인', '메시지', '센터', '시', '의료원', '붕괴', '기자실', '보고서', '소폭', '라이벌', '노조', '내년도', '견제', '앵커',
             '논설위원', '수락', '리서치', '타임스', '무죄', '뉴시스', '도', '조사', '상당수', '지난달', '마다', '주', '가운데', '개방',
            '연평균', '고속버스', '평균', '관계자', '고교', '연면적', '참가자', '당시']

final_list = []

for i in range(len(data)):
    line = data.iloc[i, 4]  # '단어난이도' 열에 접근
    line_without_nan = [(word, num) for word, num in line if not pd.isna(num)]  # tuple 내에 nan 있으면 제거
    sorted_result = sorted(line_without_nan, key=lambda x: x[1], reverse=True)  # 최종 난이도 기준으로 내림차순 정렬

    unique_values = set()
    unique_result = [(word, num) for word, num in sorted_result if word not in unique_values and not unique_values.add(word)]  # 중복 제거

    # Remove stopwords
    unique_result = [(word, num) for word, num in unique_result if word not in stopwords]

    final_list.append(unique_result)

# 샘플데이터에 추가 및 확인
data['단어난이도'] = final_list

#------------------------------# 최종난이도만 뽑아내기
num_list = []

for i in range(len(data)):
    line = data.iloc[i, 4] # '단어난이도' 열에 접근
    for (_, num) in line:
        num_list.append(num)

#------------------------------# 최종난이도 상위 6개만 추출
top6_list = []

for i in range(len(data)):
    line = data.iloc[i, 4]
    lbyl_list = []
    for (word, num) in line:
        lbyl_list.append(word)
        if len(lbyl_list) == 6:
            break
    top6_list.append(lbyl_list)

# 데이터에 추가
data['top6'] = top6_list

#------------------------------# 단어장 제공 단어


#------------------------------# 최종난이도 상위 6개만 추출 새로운 리스트를 담을 변수
final_result = []

for i in range(len(data)): # 데이터셋의 모든 행에 대해
    list1 = data.iloc[i, 4] # '단어난이도' 열에 접근
                           
    단어난이도_list = []

    for tup1 in list1:        
        # 튜플에서 숫자 추출
        단어난이도 = tup1[1]
        단어난이도_list.append(단어난이도)
            
    # 결과 리스트에 새 튜플리스트들을 축적!
    단어난이도_sum = sum(단어난이도_list)/len(단어난이도_list)
    final_result.append(단어난이도_sum)

# 데이터에 추가
data['명사난이도 평균 점수'] = final_result

# 전체 실행 시간 출력
total_time = time.time() - start_time
print(f"Final Score Calculated elapsed time: {total_time:.2f} seconds")

In [ ]:
# 불용어 지정
stopwords = ['요구', '특파원', '참석자', '희생자', '기자', '지난해', '양국', '갑', '을', '중대형', '승용차', '이듬해', '핸드볼',
             '국가', '연합뉴스', '당국', '지난해', '기업', '상승세', '닷새', '누리집', '꼴찌', '사망자', '이날', '대통령', '지역',
             '시인', '메시지', '센터', '시', '의료원', '붕괴', '기자실', '보고서', '소폭', '라이벌', '노조', '내년도', '견제', '앵커',
             '논설위원', '수락', '리서치', '타임스', '무죄', '뉴시스', '도', '조사', '상당수', '지난달', '마다', '주']